In [1]:
# install libraries
!pip install rasterio
!pip install tqdm
!pip install multiprocess

     |████████████████████████████████| 19.3 MB 1.3 MB/s 


In [2]:
# import libraries
import os
import rasterio
from tqdm.auto import tqdm
from multiprocess import Pool,cpu_count
from pathlib import Path

In [3]:
# downlaod large geotiff
!gdown --id 13nA0BA3N0B43ItR-aqXaFzplxnliZNMa

Downloading...
From: https://drive.google.com/uc?id=13nA0BA3N0B43ItR-aqXaFzplxnliZNMa
To: /content/202109100217_S2A_MSI_ESA_50HNH_432_chunks.tif
100% 94.7M/94.7M [00:00<00:00, 183MB/s]


In [4]:
# point variable to data
raster_path = '/content/202109100217_S2A_MSI_ESA_50HNH_432_chunks.tif'
os.path.isfile(raster_path)

True

In [5]:
# make folder for tiles
output_folder = '/content/tiles'
Path(output_folder).mkdir(parents=True, exist_ok=True)

In [6]:
# set some variables
size = [500,500] #[x,y]
output_compression = 'JPEG'#'lZW'
output_format = 'GTiff'
output_extention = '.tif'

In [7]:
# open metadata and check it out
with rasterio.open(raster_path) as open_raster:
    input_meta = open_raster.meta
input_meta

{'count': 4,
 'crs': CRS.from_epsg(4326),
 'driver': 'GTiff',
 'dtype': 'uint8',
 'height': 13363,
 'nodata': None,
 'transform': Affine(8.983152841677949e-05, 0.0, 116.999782326,
       0.0, -7.453201781037179e-05, -33.433410993),
 'width': 13302}

In [8]:
# make a list of tiles by looping over raster height and width
tiles = []
left = 0
col = 0
while left < input_meta['width']:
    top = 0
    row = 0
    while top < input_meta['height']:
        name = 'tile_'+str(row)+'_'+str(col)+output_extention
        export_path = os.path.join(output_folder,name)
        tiles.append([left,top,export_path])
        top += size[1]
        row += 1
    left += size[0]
    col += 1
print(len(tiles))
tiles[0]

729


[0, 0, '/content/tiles/tile_0_0.tif']

In [9]:
cores = cpu_count()
cores

2

In [10]:
# func to split list into n chunks
def split(a, n):
    k, m = divmod(len(a), n)
    return (a[i*k+min(i, m):(i+1)*k+min(i+1, m)] for i in range(n))

In [11]:
# func that open raster and exports tiles
def tile_cutter(chunks):
    with rasterio.open(raster_path) as src:
        for tile in chunks:
            win = rasterio.windows.Window(tile[0], tile[1], size[0], size[1])
            win_data = src.read(window=win)
            win_transform = src.window_transform(win)
            meta = src.meta
            meta['compress'] = output_compression
            meta['driver'] = output_format
            meta['transform'] = win_transform
            meta['width'] = win_data.shape[2]
            meta['height'] = win_data.shape[1]
            with rasterio.open(tile[2], 'w', **meta) as dst:
                dst.write(win_data)
            win_data = None

In [ ]:
with Pool(cores) as p:
    list(tqdm(p.imap(tile_cutter, list(split(tiles,cores))), total=len(list(split(tiles,cores)))))

  0%|          | 0/2 [00:00<?, ?it/s]